# Verifica dados do Farol com novos IDs e regional

---

- (extra) Verificando estado sem Rt
- `cities/farolcovid`: Consistência de alertas de cidades na nova tabela
- `cities/simulacovid`: Mudança do fillna de subnotificação para regional (antes estado)
- Cálculos de indicadores para regionais

In [1]:
import pandas as pd

## Rt de estados

- MT não possui Rt calculado mais!

In [2]:
df = pd.read_csv("http://45.55.43.231:7000/br/states/rt")
ref = pd.read_csv("http://45.55.43.231:7000/br/places/ids")

ref[ref["state_num_id"].isin(df["state_num_id"]) == False][["state_num_id", "state_id"]].drop_duplicates()

,state_num_id,state_id
2277,51,MT


In [3]:
cases = pd.read_csv("http://45.55.43.231:7000/br/cities/cases/full")
mt = cases[(cases["state_num_id"] == 51) & (cases["is_last"] == True)]

mt.groupby(["state_num_id", "last_updated"]).sum()[["confirmed_cases", "active_cases", "deaths"]]

,,confirmed_cases,active_cases,deaths
state_num_id,last_updated,,,
51,2020-07-12,28658.0,27729.0,1060.0


## Alertas de cidades no FarolCovid 

- Passamos de 3152 para 3040 municípios com alerta
- 137 municípios não possuem mais alerta: subnotificação não é a nível município, 4 não possuem mais Rt
- 25 novos municípios com alerta: não era reconhecido que tinham Rt calculado

In [40]:
df = pd.read_csv("http://45.55.43.231:7000/br/cities/farolcovid/main")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 47 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   city_id                          5570 non-null   int64  
 1   state_num_id                     5570 non-null   int64  
 2   state_id                         5570 non-null   object 
 3   state_name                       5570 non-null   object 
 4   health_region_id                 5570 non-null   int64  
 5   health_region_name               5570 non-null   object 
 6   city_name                        5570 non-null   object 
 7   population                       5570 non-null   int64  
 8   last_updated_number_beds         5570 non-null   object 
 9   author_number_beds               5570 non-null   object 
 10  number_beds                      5570 non-null   int64  
 11  last_updated_number_ventilators  5570 non-null   object 
 12  author_number_ventil

In [41]:
ref = pd.read_csv("http://datasource.coronacidades.org/br/cities/farolcovid/main")
ref.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 45 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   city_id                          5570 non-null   int64  
 1   city_name                        5570 non-null   object 
 2   state_id                         5570 non-null   object 
 3   state_name                       5570 non-null   object 
 4   population                       5570 non-null   int64  
 5   health_system_region             5561 non-null   object 
 6   last_updated_number_beds         5570 non-null   object 
 7   author_number_beds               5570 non-null   object 
 8   number_beds                      5570 non-null   int64  
 9   last_updated_number_ventilators  5570 non-null   object 
 10  author_number_ventilators        5570 non-null   object 
 11  number_ventilators               5570 non-null   int64  
 12  confirmed_cases     

### Distribuição de alertas

In [42]:
print("Total:", ref["overall_alert"].count())
ref["overall_alert"].value_counts()

Total: 3152


alto     2444
medio     661
baixo      47
Name: overall_alert, dtype: int64

In [43]:
print("Total:", df["overall_alert"].count())
df["overall_alert"].value_counts()

Total: 3040


alto     2314
medio     667
baixo      59
Name: overall_alert, dtype: int64

In [44]:
ref[ref["overall_alert"] == "baixo"]["subnotification_rate"].max()

0.4980069701681384

In [45]:
df[df["overall_alert"] == "baixo"]["subnotification_rate"].max()

0.4980069701681384

#### Casos com alerta diferente

In [46]:
alert = df[["city_id", "overall_alert"]].copy()
alert = alert.merge(ref[["city_id", "overall_alert"]], on="city_id", suffixes=('_new', '_old')).fillna("-")

alert = alert[alert["overall_alert_new"] != alert["overall_alert_old"]]
alert

,city_id,overall_alert_new,overall_alert_old
2,1100031,-,alto
24,1100346,medio,alto
65,1200393,-,alto
90,1301001,alto,-
114,1302900,baixo,-
...,...,...,...
5483,5214838,alto,-
5506,5217203,baixo,alto
5517,5218607,-,alto
5523,5219100,medio,alto


#### Novos alertas nulos: subnotificação não é da cidade

In [47]:
nulos = df[(df["city_id"].isin(alert["city_id"])) & (df["overall_alert"].isnull())]
nulos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 137 entries, 2 to 5517
Data columns (total 47 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   city_id                          137 non-null    int64  
 1   state_num_id                     137 non-null    int64  
 2   state_id                         137 non-null    object 
 3   state_name                       137 non-null    object 
 4   health_region_id                 137 non-null    int64  
 5   health_region_name               137 non-null    object 
 6   city_name                        137 non-null    object 
 7   population                       137 non-null    int64  
 8   last_updated_number_beds         137 non-null    object 
 9   author_number_beds               137 non-null    object 
 10  number_beds                      137 non-null    int64  
 11  last_updated_number_ventilators  137 non-null    object 
 12  author_number_ventila

In [52]:
nulos["city_notification_place_type"].value_counts()

health_region    132
city               4
state              1
Name: city_notification_place_type, dtype: int64

In [54]:
ref[(ref["city_id"].isin(nulos["city_id"]))]["subnotification_place_type"].value_counts()

city    137
Name: subnotification_place_type, dtype: int64

In [56]:
compare = nulos[nulos["city_notification_place_type"] == "city"]
compare.sort_values("city_id")

,city_id,state_num_id,state_id,state_name,health_region_id,health_region_name,city_name,population,last_updated_number_beds,author_number_beds,number_beds,last_updated_number_ventilators,author_number_ventilators,number_ventilators,confirmed_cases,active_cases,deaths,notification_rate,state_notification_rate,city_notification_place_type,last_updated_subnotification,subnotification_rate,subnotification_rank,subnotification_classification,last_updated_rt,rt_10days_ago_low,rt_10days_ago_high,rt_10days_ago_most_likely,rt_17days_ago_low,rt_17days_ago_high,rt_17days_ago_most_likely,rt_classification,rt_10days_ago_avg,rt_17days_ago_avg,rt_ratio_week_avg,rt_growth,last_updated_inloco,inloco_today_7days_avg,inloco_last_week_7days_avg,inloco_ratio_week_avgs,inloco_growth,dday_beds_best,dday_beds_worst,dday_classification,dday_beds_best_months,overall_alert,data_last_refreshed
1097,2402006,24,RN,Rio Grande do Norte,24004,4ª Região de Saúde - Caicó,Caicó,67952,2020-05-01,DataSUS,141,2020-05-01,DataSUS,21,738.0,321.0,14.0,0.986054,0.551276,city,2020-07-27 20:47:57,0.013946,28.0,bom,2020-07-27 19:57:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-27 18:01:52,0.388038,0.385923,1.005481,estabilizando,91.0,91.0,bom,3.0,NaN,2020-07-27 20:50:43
1604,2612406,26,PE,Pernambuco,26003,Caruaru,Sanharó,26462,2020-05-01,DataSUS,5,2020-05-01,DataSUS,0,297.0,221.0,7.0,0.715510,0.271265,city,2020-07-27 20:47:57,0.284490,30.0,bom,2020-07-27 19:57:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-27 18:01:52,0.395796,0.441055,0.897385,piorando,1.0,1.0,ruim,1.0,NaN,2020-07-27 20:50:43
3496,3520103,35,SP,São Paulo,35083,Alta Mogiana,Igarapava,30432,2020-05-01,DataSUS,42,2020-05-01,DataSUS,2,154.0,57.0,3.0,0.940000,0.432951,city,2020-07-27 20:47:57,0.060000,93.0,bom,2020-07-27 19:57:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-27 18:01:52,0.427101,0.410839,1.039581,estabilizando,91.0,91.0,bom,3.0,NaN,2020-07-27 20:50:43
5345,5201702,52,GO,Goiás,52009,Oeste I,Aragarças,20118,2020-05-01,DataSUS,25,2020-05-01,DataSUS,2,254.0,177.0,3.0,1.000000,0.783658,city,2020-07-27 20:47:57,0.000000,4.0,bom,2020-07-27 19:57:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-27 18:01:52,0.416352,0.401080,1.038077,estabilizando,1.0,1.0,ruim,1.0,NaN,2020-07-27 20:50:43


In [57]:
ref[(ref["city_id"].isin(compare["city_id"]))].sort_values("city_id")

,city_id,city_name,state_id,state_name,population,health_system_region,last_updated_number_beds,author_number_beds,number_beds,last_updated_number_ventilators,author_number_ventilators,number_ventilators,confirmed_cases,active_cases,deaths,notification_rate,state_notification_rate,last_updated_subnotification,subnotification_place_type,subnotification_rate,subnotification_rank,subnotification_classification,last_updated_rt,rt_10days_ago_low,rt_10days_ago_high,rt_10days_ago_most_likely,rt_17days_ago_low,rt_17days_ago_high,rt_17days_ago_most_likely,rt_classification,rt_10days_ago_avg,rt_17days_ago_avg,rt_ratio_week_avg,rt_growth,last_updated_inloco,inloco_today_7days_avg,inloco_last_week_7days_avg,inloco_ratio_week_avgs,inloco_growth,dday_beds_best,dday_beds_worst,dday_classification,dday_beds_best_months,overall_alert,data_last_refreshed
1097,2402006,Caicó,RN,Rio Grande do Norte,67952,IV ETAM,2020-05-01,DataSUS,141,2020-05-01,DataSUS,21,738.0,321.0,14.0,0.986054,0.551276,2020-07-27 18:10:03,city,0.013946,28.0,bom,2020-07-27 19:37:29,1.04,1.44,1.26,1.03,1.47,1.28,ruim,1.284286,1.314286,0.977174,estabilizando,2020-07-27 18:12:08,0.388038,0.385923,1.005481,estabilizando,91.0,28.0,bom,3.0,alto,2020-07-27 19:38:42
1604,2612406,Sanharó,PE,Pernambuco,26462,Caruaru,2020-05-01,DataSUS,5,2020-05-01,DataSUS,0,297.0,221.0,7.0,0.715510,0.271265,2020-07-27 18:10:03,city,0.284490,30.0,bom,2020-07-27 19:37:29,1.14,1.55,1.36,1.12,1.59,1.37,ruim,1.390000,1.244286,1.117107,piorando,2020-07-27 18:12:08,0.395796,0.441055,0.897385,piorando,1.0,1.0,ruim,1.0,alto,2020-07-27 19:38:42
3496,3520103,Igarapava,SP,São Paulo,30432,Franca,2020-05-01,DataSUS,42,2020-05-01,DataSUS,2,154.0,57.0,3.0,0.940000,0.432951,2020-07-27 18:10:03,city,0.060000,93.0,bom,2020-07-27 19:37:29,0.88,1.53,1.23,0.85,1.55,1.23,ruim,1.235714,1.257143,0.982955,estabilizando,2020-07-27 18:12:08,0.427101,0.410839,1.039581,estabilizando,91.0,38.0,bom,3.0,alto,2020-07-27 19:38:42
5345,5201702,Aragarças,GO,Goiás,20118,Oeste I,2020-05-01,DataSUS,25,2020-05-01,DataSUS,2,254.0,177.0,3.0,1.000000,0.783658,2020-07-27 18:10:03,city,0.000000,4.0,bom,2020-07-27 19:37:29,1.13,2.03,1.60,1.13,2.32,1.75,ruim,1.694286,1.772857,0.955681,estabilizando,2020-07-27 18:12:08,0.416352,0.401080,1.038077,estabilizando,1.0,1.0,ruim,1.0,alto,2020-07-27 19:38:42


#### Antigos alertas nulos: não tinham Rt

In [58]:
old_nulos = ref[(ref["city_id"].isin(alert["city_id"])) & (ref["overall_alert"].isnull())]
old_nulos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 90 to 5483
Data columns (total 45 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   city_id                          25 non-null     int64  
 1   city_name                        25 non-null     object 
 2   state_id                         25 non-null     object 
 3   state_name                       25 non-null     object 
 4   population                       25 non-null     int64  
 5   health_system_region             25 non-null     object 
 6   last_updated_number_beds         25 non-null     object 
 7   author_number_beds               25 non-null     object 
 8   number_beds                      25 non-null     int64  
 9   last_updated_number_ventilators  25 non-null     object 
 10  author_number_ventilators        25 non-null     object 
 11  number_ventilators               25 non-null     int64  
 12  confirmed_cases      

In [59]:
old_nulos["rt_ratio_week_avg"].unique()

array([nan])

In [60]:
df[df["city_id"].isin(old_nulos["city_id"]) & (df["rt_ratio_week_avg"].isnull())]

,city_id,state_num_id,state_id,state_name,health_region_id,health_region_name,city_name,population,last_updated_number_beds,author_number_beds,number_beds,last_updated_number_ventilators,author_number_ventilators,number_ventilators,confirmed_cases,active_cases,deaths,notification_rate,state_notification_rate,city_notification_place_type,last_updated_subnotification,subnotification_rate,subnotification_rank,subnotification_classification,last_updated_rt,rt_10days_ago_low,rt_10days_ago_high,rt_10days_ago_most_likely,rt_17days_ago_low,rt_17days_ago_high,rt_17days_ago_most_likely,rt_classification,rt_10days_ago_avg,rt_17days_ago_avg,rt_ratio_week_avg,rt_growth,last_updated_inloco,inloco_today_7days_avg,inloco_last_week_7days_avg,inloco_ratio_week_avgs,inloco_growth,dday_beds_best,dday_beds_worst,dday_classification,dday_beds_best_months,overall_alert,data_last_refreshed


## Rt cidades

- Valores existem para >27 dias as cidades que não tinham Rt na tabela do Farol oficial! Algo devia estar errado no código...

In [61]:
df = pd.read_csv("http://45.55.43.231:7000/br/cities/rt")
ref = pd.read_csv("http://45.55.43.231:7000/br/cities/rt")

In [64]:
ref[ref["city_id"].isin(old_nulos["city_id"])]["city_id"].value_counts().min()

27

In [65]:
df[df["city_id"].isin(old_nulos["city_id"])]["city_id"].value_counts().min()

27

## SimulaCovid

In [21]:
df = pd.read_csv("http://45.55.43.231:7000/br/cities/simulacovid/main")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 32 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   country_iso                            5570 non-null   object 
 1   country_name                           5570 non-null   object 
 2   state_num_id                           5570 non-null   int64  
 3   state_id                               5570 non-null   object 
 4   state_name                             5570 non-null   object 
 5   health_region_id                       5570 non-null   int64  
 6   health_region_name                     5570 non-null   object 
 7   city_id                                5570 non-null   int64  
 8   city_name                              5570 non-null   object 
 9   population                             5570 non-null   int64  
 10  last_updated_number_beds               5570 non-null   object 
 11  auth

In [22]:
ref = pd.read_csv("http://datasource.coronacidades.org/br/cities/simulacovid/main")
ref.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 28 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   country_iso                      5570 non-null   object 
 1   country_name                     5570 non-null   object 
 2   state_id                         5570 non-null   object 
 3   state_name                       5570 non-null   object 
 4   city_id                          5570 non-null   int64  
 5   city_name                        5570 non-null   object 
 6   population                       5570 non-null   int64  
 7   health_system_region             5561 non-null   object 
 8   last_updated_number_beds         5570 non-null   object 
 9   author_number_beds               5570 non-null   object 
 10  last_updated_number_ventilators  5570 non-null   object 
 11  author_number_ventilators        5570 non-null   object 
 12  number_ventilators  

### Diferença das taxas de notificação

In [23]:
compare = df[["city_id", "notification_rate", 
              "city_notification_place_type"]].merge(ref[["city_id", "notification_rate", "state_notification_rate"]], 
                                                    on="city_id", suffixes=('_new', '_old'))

compare = compare[compare["notification_rate_new"] != compare["notification_rate_old"]]
compare

,city_id,notification_rate_new,city_notification_place_type,notification_rate_old,state_notification_rate
13,1200393,1.000000,health_region,0.759664,0.759664
22,2700102,1.000000,health_region,0.725699,0.725699
40,2701803,1.000000,health_region,0.725699,0.725699
41,2701902,0.758312,health_region,0.725699,0.725699
48,2702504,1.000000,health_region,0.725699,0.725699
...,...,...,...,...,...
5555,1720309,0.845082,health_region,1.000000,1.000000
5560,1720903,0.826667,health_region,1.000000,1.000000
5561,1720937,0.826667,health_region,1.000000,1.000000
5566,1721257,0.894326,health_region,1.000000,1.000000


In [24]:
compare["city_notification_place_type"].value_counts()

health_region    1886
Name: city_notification_place_type, dtype: int64

In [25]:
t = compare[compare["city_notification_place_type"].isnull()]

print("Novos municípios com notificação:", len(t))
t

Novos municípios com notificação: 127


,city_id,notification_rate_new,city_notification_place_type,notification_rate_old,state_notification_rate
256,2904407,1.000000,NaN,0.878400,0.878400
276,2906105,1.000000,NaN,0.878400,0.878400
338,2911253,1.000000,NaN,0.878400,0.878400
579,2930758,1.000000,NaN,0.878400,0.878400
932,5204656,1.000000,NaN,0.783658,0.783658
...,...,...,...,...,...
4824,3503356,0.394273,NaN,0.432951,0.432951
4933,3513306,0.684396,NaN,0.432951,0.432951
4970,3516101,0.684396,NaN,0.432951,0.432951
5268,3543105,0.515471,NaN,0.432951,0.432951


In [26]:
# todas antes eram do estado
compare[compare["state_notification_rate"] != compare["notification_rate_old"]]

,city_id,notification_rate_new,city_notification_place_type,notification_rate_old,state_notification_rate


### Verificando tabela de casos

- 1 dia com morte zerada = 1 semana sem taxa de notificação

In [27]:
pd.options.display.max_columns=999

# exemplo:
cases[cases["city_id"] == 1100031]

,city_name,city_id,last_updated,epidemiological_week,estimated_population_2019,is_last,is_repeated,confirmed_cases,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,deaths,order_for_place,place_type,state_id,daily_cases,new_deaths,state_name,state_num_id,health_region_name,health_region_id,infectious_period_cases,city_notification_place_type,health_region_notification_place_type,notification_rate,city_notification_rate,health_region_notification_rate,state_notification_rate,active_cases,data_last_refreshed
194,Cabixi,1100031,2020-05-02,18,5312.0,False,False,1.0,18.82530,2020-05-02,0.0000,0.0,1,city,RO,1.0,0.0,Rondônia,11,Cone Sul,11006,1.0,state,state,0.715641,NaN,0.715641,0.715641,1.0,2020-07-27 20:47:57
195,Cabixi,1100031,2020-05-03,19,5312.0,False,False,1.0,0.00000,2020-05-03,0.0000,0.0,2,city,RO,0.0,0.0,Rondônia,11,Cone Sul,11006,1.0,state,state,0.684133,NaN,0.684133,0.684133,1.0,2020-07-27 20:47:57
196,Cabixi,1100031,2020-05-04,19,5312.0,False,False,0.0,0.00000,2020-05-04,0.0000,0.0,3,city,RO,-1.0,0.0,Rondônia,11,Cone Sul,11006,0.0,state,state,0.645771,NaN,0.645771,0.645771,0.0,2020-07-27 20:47:57
197,Cabixi,1100031,2020-05-05,19,5312.0,False,False,0.0,0.00000,2020-05-05,0.0000,0.0,4,city,RO,0.0,0.0,Rondônia,11,Cone Sul,11006,0.0,state,state,0.612698,NaN,0.612698,0.612698,0.0,2020-07-27 20:47:57
198,Cabixi,1100031,2020-05-06,19,5312.0,False,False,0.0,0.00000,2020-05-06,0.0000,0.0,5,city,RO,0.0,0.0,Rondônia,11,Cone Sul,11006,0.0,state,state,0.605975,NaN,0.605975,0.605975,0.0,2020-07-27 20:47:57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,Cabixi,1100031,2020-07-22,30,5312.0,False,False,35.0,658.88554,2020-07-22,0.0571,2.0,82,city,RO,1.0,0.0,Rondônia,11,Cone Sul,11006,20.0,health_region,health_region,1.000000,NaN,1.000000,0.849264,20.0,2020-07-27 20:47:57
276,Cabixi,1100031,2020-07-23,30,5312.0,False,False,37.0,696.53614,2020-07-23,0.0811,3.0,83,city,RO,2.0,1.0,Rondônia,11,Cone Sul,11006,21.0,health_region,health_region,1.000000,NaN,1.000000,0.852534,21.0,2020-07-27 20:47:57
277,Cabixi,1100031,2020-07-24,30,5312.0,True,False,37.0,696.53614,2020-07-24,0.0811,3.0,84,city,RO,0.0,0.0,Rondônia,11,Cone Sul,11006,18.0,health_region,health_region,1.000000,NaN,1.000000,0.855654,18.0,2020-07-27 20:47:57
278,Cabixi,1100031,2020-07-25,30,5312.0,False,True,37.0,696.53614,2020-07-24,0.0811,3.0,85,city,RO,0.0,0.0,Rondônia,11,Cone Sul,11006,18.0,health_region,health_region,1.000000,NaN,1.000000,0.858775,18.0,2020-07-27 20:47:57


## Alertas de estados no FarolCovid 

- PI passou para Baixo
- MT não possui Rt

In [69]:
df = pd.read_csv("http://45.55.43.231:7000/br/states/farolcovid/main")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 37 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   state_num_id                    27 non-null     int64  
 1   state_id                        27 non-null     object 
 2   state_name                      27 non-null     object 
 3   population                      27 non-null     int64  
 4   confirmed_cases                 27 non-null     float64
 5   active_cases                    27 non-null     float64
 6   deaths                          27 non-null     float64
 7   notification_rate               27 non-null     float64
 8   number_beds                     27 non-null     int64  
 9   number_ventilators              27 non-null     int64  
 10  last_updated_subnotification    27 non-null     object 
 11  subnotification_rate            27 non-null     float64
 12  subnotification_rank            27 non

In [70]:
ref = pd.read_csv("http://datasource.coronacidades.org/br/states/farolcovid/main")
ref.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 38 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   state_id                        27 non-null     object 
 1   state_num_id                    27 non-null     int64  
 2   state_name                      27 non-null     object 
 3   population                      27 non-null     int64  
 4   confirmed_cases                 27 non-null     float64
 5   active_cases                    27 non-null     float64
 6   deaths                          27 non-null     float64
 7   notification_rate               27 non-null     float64
 8   number_beds                     27 non-null     int64  
 9   number_ventilators              27 non-null     int64  
 10  last_updated_subnotification    27 non-null     object 
 11  subnotification_place_type      0 non-null      float64
 12  subnotification_rate            27 non

In [71]:
df["overall_alert"].value_counts()

alto     11
baixo     9
medio     6
Name: overall_alert, dtype: int64

In [72]:
ref["overall_alert"].value_counts()

alto     13
baixo     8
medio     6
Name: overall_alert, dtype: int64

In [87]:
diff = list(set(df[df["overall_alert"] == "baixo"]["state_id"]) - set(ref[ref["overall_alert"] == "baixo"]["state_id"]))
diff

['PI']

In [88]:
df.loc[df[df["state_id"] == diff[0]].index[0]]

state_num_id                                       22
state_id                                           PI
state_name                                      Piauí
population                                    3273227
confirmed_cases                                 46584
active_cases                                    19567
deaths                                           1259
notification_rate                            0.729756
number_beds                                      4991
number_ventilators                                497
last_updated_subnotification      2020-07-27 20:47:57
subnotification_rate                         0.270244
subnotification_rank                               17
subnotification_classification                    bom
last_updated_rt                   2020-07-27 20:49:14
rt_10days_ago_low                                0.94
rt_10days_ago_high                               1.03
rt_10days_ago_most_likely                        0.99
rt_17days_ago_low           

In [89]:
ref.loc[ref[ref["state_id"] == diff[0]].index[0]]

state_id                                           PI
state_num_id                                       22
state_name                                      Piauí
population                                    3273227
confirmed_cases                                 46584
active_cases                                    19843
deaths                                           1259
notification_rate                            0.729756
number_beds                                      4991
number_ventilators                                497
last_updated_subnotification      2020-07-27 18:10:03
subnotification_place_type                        NaN
subnotification_rate                         0.270244
subnotification_rank                               17
subnotification_classification                    bom
last_updated_rt                   2020-07-25 18:10:57
rt_10days_ago_low                                   1
rt_10days_ago_high                               1.09
rt_10days_ago_most_likely   

## Alertas de regionais no FarolCovid 

- 9 regionais sem classificação: 2 não possuem Rt, 7 não possuem projeção de capacidade

In [96]:
df = pd.read_csv("http://45.55.43.231:7000/br/health_region/farolcovid/main")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 0 to 449
Data columns (total 40 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   health_region_id                       450 non-null    int64  
 1   state_num_id                           450 non-null    int64  
 2   state_id                               450 non-null    object 
 3   state_name                             450 non-null    object 
 4   health_region_name                     450 non-null    object 
 5   health_region_notification_place_type  450 non-null    object 
 6   population                             450 non-null    int64  
 7   confirmed_cases                        450 non-null    float64
 8   active_cases                           450 non-null    float64
 9   deaths                                 450 non-null    float64
 10  notification_rate                      450 non-null    float64
 11  number

In [97]:
df["overall_alert"].value_counts()

alto     256
medio    156
baixo     29
Name: overall_alert, dtype: int64

In [98]:
nulos = df[df["overall_alert"].isnull()]
nulos

,health_region_id,state_num_id,state_id,state_name,health_region_name,health_region_notification_place_type,population,confirmed_cases,active_cases,deaths,notification_rate,number_beds,number_ventilators,last_updated_subnotification,subnotification_rate,subnotification_rank,subnotification_classification,last_updated_rt,rt_10days_ago_low,rt_10days_ago_high,rt_10days_ago_most_likely,rt_17days_ago_low,rt_17days_ago_high,rt_17days_ago_most_likely,rt_classification,rt_10days_ago_avg,rt_17days_ago_avg,rt_ratio_week_avg,rt_growth,last_updated_inloco,inloco_today_7days_avg,inloco_last_week_7days_avg,inloco_ratio_week_avgs,inloco_growth,dday_beds_best,dday_beds_worst,dday_classification,dday_beds_best_months,overall_alert,data_last_refreshed
182,31064,31,MG,Minas Gerais,Araçuaí,state,80416,37.0,18.0,0.0,0.929571,134,9,2020-07-27 20:47:57,0.070429,NaN,NaN,2020-07-27 18:00:08,0.78,2.46,1.63,0.68,2.55,1.61,ruim,1.602857,1.681429,0.953271,estabilizando,2020-07-27 18:01:52,0.403374,0.405875,0.993839,estabilizando,NaN,NaN,NaN,NaN,NaN,2020-07-27 20:51:46
192,31091,31,MG,Minas Gerais,Cassia,state,50445,124.0,75.0,0.0,0.929571,65,4,2020-07-27 20:47:57,0.070429,NaN,NaN,2020-07-27 18:00:08,0.97,2.08,1.56,0.89,2.18,1.55,ruim,1.505714,1.641429,0.917319,estabilizando,2020-07-27 18:01:52,0.422745,0.410015,1.031047,estabilizando,NaN,NaN,NaN,NaN,NaN,2020-07-27 20:51:46
196,31050,31,MG,Minas Gerais,Coração de Jesus,state,43150,54.0,38.0,0.0,0.929571,25,1,2020-07-27 20:47:57,0.070429,NaN,NaN,2020-07-27 18:00:08,0.95,2.78,1.86,0.91,3.33,2.11,ruim,1.961429,2.055714,0.954135,estabilizando,2020-07-27 18:01:52,0.399613,0.379195,1.053845,estabilizando,NaN,NaN,NaN,NaN,NaN,2020-07-27 20:51:46
217,31077,31,MG,Minas Gerais,João Pinheiro,state,73522,191.0,44.0,0.0,0.929571,39,5,2020-07-27 20:47:57,0.070429,NaN,NaN,2020-07-27 18:00:08,1.09,1.80,1.47,1.38,2.29,1.87,ruim,1.604286,2.054286,0.780946,melhorando,2020-07-27 18:01:52,0.382266,0.377333,1.013071,estabilizando,NaN,NaN,NaN,NaN,NaN,2020-07-27 20:51:46
223,31076,31,MG,Minas Gerais,Manga,state,52022,57.0,40.0,0.0,0.929571,35,1,2020-07-27 20:47:57,0.070429,NaN,NaN,2020-07-27 18:00:08,0.80,2.70,1.76,0.78,3.11,1.93,ruim,1.834286,2.115714,0.866982,melhorando,2020-07-27 18:01:52,0.406694,0.407667,0.997614,estabilizando,NaN,NaN,NaN,NaN,NaN,2020-07-27 20:51:46
257,31027,31,MG,Minas Gerais,Turmalina/Minas Novas/Capelinha,state,114320,57.0,35.0,1.0,0.929571,97,5,2020-07-27 20:47:57,0.070429,NaN,NaN,2020-07-27 18:00:08,0.80,2.38,1.61,0.87,2.72,1.78,ruim,1.618571,1.904286,0.849962,melhorando,2020-07-27 18:01:52,0.383484,0.387294,0.990164,estabilizando,NaN,NaN,NaN,NaN,NaN,2020-07-27 20:51:46
264,31061,31,MG,Minas Gerais,Viçosa,state,125334,201.0,54.0,0.0,0.929571,214,25,2020-07-27 20:47:57,0.070429,NaN,NaN,2020-07-27 18:00:08,0.94,1.63,1.31,0.87,1.63,1.27,ruim,1.304286,1.250000,1.043429,estabilizando,2020-07-27 18:01:52,0.440744,0.431992,1.020261,estabilizando,NaN,NaN,NaN,NaN,NaN,2020-07-27 20:51:46
303,35114,35,SP,São Paulo,Extremo Oeste Paulista,health_region,99135,386.0,488.0,16.0,0.432421,115,12,2020-07-27 20:47:57,0.567579,55.0,insatisfatório,2020-07-27 18:00:08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-27 18:01:52,0.401628,0.390771,1.027782,estabilizando,1.0,1.0,ruim,1.0,NaN,2020-07-27 20:51:46
423,51009,51,MT,Mato Grosso,Norte Araguaia Karajá,health_region,24897,108.0,59.0,2.0,1.000000,23,1,2020-07-27 20:47:57,0.000000,3.0,bom,2020-07-27 18:00:08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-27 18:01:52,0.511297,0.482333,1.060050,estabilizando,91.0,91.0,bom,3.0,NaN,2020-07-27 20:51:46


In [103]:
cases[cases["health_region_id"].isin(nulos["health_region_id"][:-2])]["health_region_notification_place_type"].unique()

array(['state'], dtype=object)